In [ ]:
import matplotlib.pyplot as plt, numpy as np

We'd like to align two images with respect to each other.  We want to use a method similar to `skimage.feature.register_translation`, but we additionally want to get an uncertainty estimate on the alignment parameters.

As a test case to determine the properties our uncertainty should have, we use a simple case of two rectangles.  To generate a rectangle we use this function:

In [ ]:
def rectangle(W, H, x1, y1, x2, y2):
    assert 0 <= x1 < x2+1 <= W
    assert 0 <= y1 < y2+1 <= H
    result = np.zeros((H, W), dtype=float)
    result[y1:y2+1,x1:x2+1] = 1
    return result

Or in math terms:
$$R(x, y|x_1, y_1, x_2, y_2) = \left\{
\begin{array}{@{}ll@{}}
  1, & \text{if}\ x_1<=x<=x_2\ \text{and}\ y_1<=y<=y_2 \\
  0, & \text{otherwise}
\end{array}\right.$$
Also defining
$$\begin{align}w&=x_2-x_1 \\ h&=y_2-y_1\end{align}$$

To do the alignment, we maximize the cross-correlation of the function with itself.  The cross correlation is:
$$C(d\vec{x})=\int_0^W dx \int_0^H dy I_1(\vec x) I_2(\vec x+d\vec x)$$

In [ ]:
def crosscorr(a, b):
    A = np.fft.fft2(a)
    B = np.fft.fft2(b)
    C = A * np.conj(B)
    c = np.real(np.fft.ifft2(C))
    c = np.roll(c, c.shape[0]//2, axis=0)
    c = np.roll(c, c.shape[1]//2, axis=1)
    return c

To start with, let's say the images are two identical rectangles, and we'll also add a variable $A$ with units of intensity to allow for normalization:
$$
\begin{align}
I_1(x, y) &= I_2(x, y) = AR(x, y | x_1, y_1, x_2, y_2)
\end{align}
$$

In [ ]:
I1 = I2 = rectangle(100, 100, 40, 45, 60, 55)
plt.imshow(I1); None

So then:
$$
C(d\vec{x})=\left\{
\begin{array}{@{}ll@{}}
A^2(w-|dx|)(h-|dy|), & \text{if}\ dx<w\ \text{and}\ dy<h \\
0, & \text{otherwise}
\end{array}\right.
$$

In [ ]:
C = crosscorr(I1, I2)
plt.imshow(C, extent=[-50, 50, -50, 50]); None

This is the product of triangles in the $x$ and $y$ directions.  Note that we _can't_ use the width of this triangle as an error estimate.  The width of the triangle (however exactly you define that) is proportional to the widrh of the rectangle, so for example if we made a bigger rectangle along $x$, we get a wider triangle along x.

In [ ]:
tmp = rectangle(100, 100, 20, 45, 80, 55)
plt.imshow(tmp)
plt.imshow(crosscorr(tmp, tmp), extent=[-50, 50, -50, 50]); None

This isn't right: statistically, we expect $\delta(dx)\sim1/\sqrt{w}$, $\delta(dy)\sim1/\sqrt{h}$.  We'd also like the intensity to play in somehow: a more intense pixel provides a stronger constraint than a less intense one.

The second derivatives of the function at the peak look useful.  For example
$$\frac{\partial C}{\partial x^\pm}(0, 0)=\mp A^2 h$$
$$\frac{\partial^2C}{\partial x^2}(0,0)=-2A^2h\delta(x)$$
The inverse square root of this is also proportional to the things we want it to be proportional to and has units of $\frac{1}{\text{intensity}}$.  Hessian matrices often show up in the denominator of errors, so that's consistent with our general expectations.

In the numerator, we need something with units of $\text{intensity}\cdot\text{length}$.  The natural choice is $\sqrt{C}$.

So, at the end of the day our covariance matrix will be $$C(0,0)\left(
\begin{array}
%
\frac{\partial^2C}{\partial x^2} &
\frac{\partial^2C}{\partial x \partial y} \\
\frac{\partial^2C}{\partial x \partial y} &
\frac{\partial^2C}{\partial y^2}
\end{array}
\right)^{-1}$$

(Obviously we can't actually evaluate this in the case of rectangles because there's a delta function, but in a real case we can fit to a smooth function.)